We would've seen most of these iteration tools in the previous sections and you'll recognise that they're ***all*** **lazy iterators** and not **iterables**, making them highly efficient.

# 01 - Aggregators

#### Lecture 

Aggregators are functions that iterate through an iterable and returns a single value that (usually) takes into account every element of the iterable.

For example `min(iterable)`, `max(iterable)`, `sum(iterable)`

Also: `any(iterable)` and `all(iterable)`. Regarding these two, remember that **every** object has a truth value. The rule for all objects is the following:

**Every object has a **`True`** truth value, **except**:

- None
- False
- 0 in any numeric type (e.g. int, float etc)
- empty sequences (e.g. list, tuple, string)
- empty mapping types (e.g. dictionaries, sets)
- custom classes that implement a `__bool__` or `__len__` method that returns False or 0. If neither is present, we default to `True`.

**Definition**

Predicate: A predicate is a function that takes a single argument and returns `True` or `False`. For example, `bool()`.

We can make `all()` and `any()` more useful by first **applying a predicate** to each element of the iterable.

#### Example 1

For example, say we want to know if **every element is less that 10**:

In [18]:
l = [1, 2, 3, 4, 100, 5, 6]

pred = lambda x: x < 10

result = [pred(item) for item in l]

all(result)

False

The neater way to do this is using `map(fn, iterable)` which applies a **predicate** to each element in the iterable:

In [19]:
def pred_new(x):
    print(f'Mapping {x}', end=', ')
    return x < 10

all(map(pred_new, l))

Mapping 1, Mapping 2, Mapping 3, Mapping 4, Mapping 100, 

False

Because `map` is an iterator, it doesn't have to map every element first and then pass the result to `all()`. Instead, `all()` will lazily request a result from `map`; if it receives a `False` it will terminate immediately. This is why we don't see the final two elements (5 and 6) of `l` being printed.

If we do not want to use `map`, we should use a `generator comprehension`, *NOT* a list comprehension, so that we get the benefits of lazy evaluation.

In [20]:
result = (pred_new(item) for item in l)
all(result)

Mapping 1, Mapping 2, Mapping 3, Mapping 4, Mapping 100, 

False

#### Example 2

We have a file call `car-brands.txt`. We want to know if **every** car name is longer than 3 (4 including the \n character at the end):

Recall that `open(<file>)` (or `f`) is a lazy iterator - we don't need to loop through each line. We can pass `f` to another iterator such as `map`, and pass that output to another iterator such as `all` or `any` and they'll terminate at the right time. 

In [26]:
filename = '../Section 08 - Iteration Tools/01 - Aggregators/car-brands.txt'

with open(filename) as f:
    result = all(map(lambda row: len(row) >= 4, f))
    print(result)

True


# 02 - Slicing Iterables

Recall that we could slice sequences with `[]` notation as well as the `slice` object:

In [32]:
seq = list(range(0, 10, 1))
print(seq[2:8:2])
print(seq[slice(2, 8, 2)])

[2, 4, 6]
[2, 4, 6]


But we can slice **iterables** (including **iterators**) with `islice` from `itertools`:
```python
islice(iterable, start, stop, step)
```

- `islice` will iterate through the iterable until it has met the conditions of the slice. For example, if we only want a slice of the first 5 objects of an infinite iterable, it will raise the `StopIteration` error after the 5th element
- Recalling that all itertools are **iterators**, `islice()` will *yield* a value, not return it, and hence `islice` returns a **lazy iterator**.

In the example below `factorials()` is an infinite iterable (more specifically an iterator), not a sequence, so it cannot be sliced regularly - we need to use `islice`. 

In [46]:
import math
from itertools import islice

def factorials():
    idx = 0
    while True:
        yield math.factorial(idx)
        idx += 1

result = islice(factorials(), 2, 9, 2)
list(result)

[2, 24, 720, 40320]

We've exhausted the `islice` iterator, so we can't reuse it.

In [41]:
list(result)

[]

# 03 - Selecting and Filtering

#### `filter`

The `filter` function takes an iterable and applies a predicate to it. If the predicate returns `True`, it will retain that element; otherwise, it'll throw it away.
```python
filter(predicate, iterable)
```

The equivalent way to get the same functionality of `filter` is with the following:
```python
(item for item in iterable if pred(item))
```

In [52]:
l = [2, 1, 10, 5, 3, 6, 1, 10]
result = filter(lambda x: x < 4, l)
result

In [53]:
list(result)

[2, 1, 3, 1]

#### `filterfalse` 

This is not builtin but it's in the standard library. It does what it says on the tin:

In [55]:
from itertools import filterfalse

l = [2, 1, 10, 5, 3, 6, 1, 10]
result = filterfalse(lambda x: x < 4, l)
list(result)

[10, 5, 6, 10]

#### `compress`

This is not a compressor in the sense of say a zip archive.

It is basically a way of *filtering* one iterable, using the truthiness of items in another iterable, pairwise.

```python
data =      ['a',   'b', 'c', 'd',   'e']
              ^      ^    ^    ^      ^
              |      |    |    |      |
selectors = [True, False, 1,   0]  # None

compress(data, selectors) -> 'a', 'c'
```
Since the first and third element of `selectors` are truthy, we will only yield the first and third elements of `data`.

#### `takewhile`

```python
takewhile(pred, iterable)
```
This function returns an iterator that will yield while `predicate(item)` is Truthy. Once we run into a Falsy value, the iterator becomes exhausted.

In [61]:
from itertools import takewhile

result = takewhile(lambda x: x < 5, [1, 2, 10, 3, 4])

for i in result:
    print(i)

1
2


#### `dropwhile`

```python
dropwhile(pred, iterable)
```
This function is the inverse of the above. It returns an iterator that will start iterating and `yield` *all* remaining items unconditionally only once `predicate(item)` becomes Falsy.

In [63]:
from itertools import dropwhile

result = dropwhile(lambda x: x < 5, [1, 2, 10, 3, 4])

for i in result:
    print(i)

10
3
4


# 04 - Infinite Iterators

# 05 - Chaining and Teeing Iterators

# 06 - Mapping and Reducing

# 07 - Zipping

# 08 - Grouping

# 09 - Combinatorics